<a href="https://colab.research.google.com/github/gopalam/Geospatial_Analysis/blob/main/Geospatial_Analysis/CBG_Export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This script uses  base census data and US national geometry(geojson) provided by Safegraph to pull together select variables and attach them to the national geometry. State level shapefiles are then exported out
input: US geometry at the scale of census block group.
      **input:** Raw block group level census variables and geojson geometry
      ** output:** shapefiles exported by state


Code is interspersed with machine output as well as comments

In [ ]:
# install initial set of depdencies- for geospatial manipulation
!pip install geopandas
!pip install --upgrade pyshp
!pip install --upgrade shapely
!pip install --upgrade descartes

# Install Gdal - rtree pre-req
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git



In [ ]:
#install more dependencies for data wrangling
import pandas as pd
import psutil
from tqdm import tqdm
import warnings
import geopandas as gpd
from pathlib import Path


In [ ]:
# special libraries to allow file access
from google.colab import drive as mountGoogleDrive 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
# create file access for special libraries, this is a piece of code borrowed from Safegraph's repo.
from google.colab import drive as mountGoogleDrive 

your_name = "XX" # << If you want to customize the code, then you should edit this
################################
print(f"Hello {your_name}, to mount your google drive you will need to click on the link and copy/paste the code.")
################################
# These commands allow the notebook to read your data from your GoogleDrive
mountGoogleDrive.mount('/content/mountedDrive')
print(f"Congrats {your_name}, you mounted your google drive!")

In [ ]:
# file paths
#output_file location
filename = r"/content/mountedDrive/My Drive/census/data/shapefiles" 
#state fips code
Fips_path=r"/content/mountedDrive/My Drive/data/StateFips.csv"
# census vars
fpdata=r"/content/mountedDrive/My Drive/census/data/CBG_vars"
cbg_path=r"/content/mountedDrive/My Drive/data/cbg_geographic_data.csv"
geom_path="/content/mountedDrive/My Drive/data/cbg.geojson"
print(filename)

In [ ]:
#define the states where data is to be output
#two letter state codes are needed
# defined here is the DC-area states
st=['DC','VA','MD','DE', 'NJ','PA','NY','MA','RI','CT', 'ME', 'VT', 'NH',
    'WV','NC','KY','TN','SC','GA','FL','AL','MS','AR','LA','TX','OK',
    'WA','MT','ID','WY','OR','CA','NV','NM','UT','CO','AZ','WI','MI','OH','IL',
    'IN','MN','IA','MO','KS','NE','SD','ND','AK','HI',]
len(st)


51

In [ ]:
# read geometry. A long process. read of ~3 GB file. Can take upto 10 minutes
cbg_geos=gpd.read_file(geom_path)
cbg_geos.head()



In [ ]:
# check the unique states. There is atleast one orphan blockgroup polygon, ignore it.
cbg_geos['State'].unique()

In [ ]:
# read of basic geometry information of each block group
geo_data = pd.read_csv(cbg_path)
geo_data.head()

In [ ]:
# get cbg geometry statefips in numbers
Fips = pd.read_csv(Fips_path,dtype={'FIPS':str})
geom_fips=(cbg_geos.StateFIPS);
geom_fips=geom_fips.astype(int);    
FIPS_short=Fips[Fips['Code'].isin(st)]

FIPS_short.reset_index(inplace=True)
FIPS_short.head(100)

In [ ]:
# path to your census variables. some variables are not used
b01=r"/content/mountedDrive/My Drive/census/data/CBG_vars/cbg_b01.csv"
b03=r"/content/mountedDrive/My Drive/census/data/CBG_vars/cbg_b03.csv"
b11=r"/content/mountedDrive/My Drive/census/data/CBG_vars/cbg_b11.csv"
b15=r"/content/mountedDrive/My Drive/census/data/CBG_vars/cbg_b15.csv"
b16=r"/content/mountedDrive/My Drive/census/data/CBG_vars/cbg_b16.csv"
b19=r"/content/mountedDrive/My Drive/census/data/CBG_vars/cbg_b19.csv"
b25=r"/content/mountedDrive/My Drive/census/data/CBG_vars/cbg_b25.csv"


In [ ]:

df=pd.read_csv(b01,dtype={'census_block_group':str})

LM_pop1=df['B01001e1']
Age22t29=(df['B01001e10']+df['B01001e11']+df['B01001e34']+df['B01001e35'])/df['B01001e1']
Age30t49=(df['B01001e12'] +df['B01001e13'] +df['B01001e14']+df['B01001e15']+df['B01001e36']+
          df['B01001e37']+df['B01001e38']+df['B01001e39'])/df['B01001e1']
Age50t66=(df['B01001e16']+df['B01001e17']+df['B01001e18']+df['B01001e19']
          +df['B01001e20']+df['B01001e40']+df['B01001e41']+df['B01001e42']+df['B01001e43']
          +df['B01001e44'])/df['B01001e1']
Age67up=(df['B01001e45']+df['B01001e46']+df['B01001e47']+df['B01001e48']+df['B01001e49'])/df['B01001e1']


In [ ]:
df.head(10)

In [ ]:
#%% here are some variables created out of raw data
df=pd.read_csv(b19,dtype={'census_block_group':str})
HI_75kup=(df['B19001e13']+ df['B19001e14']+ df['B19001e15']+ df['B19001e16']+ df['B19001e17'])/(df['B19001e1'])
HI_75kup.head(10)

In [ ]:
#%% home price data
df=pd.read_csv(b25,dtype={'census_block_group':str})
HV_300up=(df['B25075e21']+ df['B25075e22']+ df['B25075e23']+ df['B25075e24']+ df['B25075e25']+ df['B25075e26']+ df['B25075e27'])/ df['B25075e1'] 
H_total=df['B25001e1']
h_frac=(df['B25024e4']+df['B25024e5']+df['B25024e6']+df['B25024e7']+df['B25024e8']+df['B25024e9'])/df['B25024e1']

In [ ]:
data=pd.DataFrame()
data=pd.DataFrame(df['census_block_group'])
data['HI_75kup']=HI_75kup
data['HV_300up']=HV_300up
data['H_total']=H_total
data['h_frac']=h_frac
data['LM_pop1']=LM_pop1
data['Age22t29']=Age22t29
data['Age30t49']=Age30t49
data['Age50t66']=Age50t66
data['Age67up']=Age67up

data.fillna(0,inplace=True)
data.head(10)


In [ ]:
data.columns

In [ ]:
data.columns=['CensusBlockGroup', 'HI_75kup', 'HV_300up', 'H_total', 'h_frac',
       'LM_pop1', 'Age22t29', 'Age30t49', 'Age50t66', 'Age67up',]

data['CensusBlockGroup']=data['CensusBlockGroup'].astype(str)


In [ ]:
cbg_geos['State'].unique()

array(['AL', 'AK', None, 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL',
       'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD',
       'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM',
       'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN',
       'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY', 'PR'], dtype=object)

In [ ]:
# merge to a different geoDF, just for safety , can fix this later.
cbg_geos2=cbg_geos.merge(data,on="CensusBlockGroup")


In [ ]:
p=cbg_geos2.copy()
del p['geometry']
p['State'].unique()

array(['AL', 'AK', None, 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL',
       'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD',
       'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM',
       'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN',
       'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY', 'PR'], dtype=object)

In [ ]:
cbg_geos=cbg_geos2.copy()

In [ ]:
cbg_geos.head(10)

,StateFIPS,CountyFIPS,TractCode,BlockGroup,CensusBlockGroup,State,County,ClassCode,geometry,HI_75kup,HV_300up,H_total,h_frac,LM_pop1,Age22t29,Age30t49,Age50t66,Age67up,spn_18t64,hisp_wht,hisp_lat,hisp_hous
0,01,081,041600,1,010810416001,AL,Lee County,H1,"MULTIPOLYGON (((-85.37282 32.63424, -85.37275 ...",0.280216,0.000000,1479,0.536173,3182,0.185104,0.300126,0.177876,0.049654,0.007703,0.008485,0.034884,0.020785
1,01,081,041600,2,010810416002,AL,Lee County,H1,"MULTIPOLYGON (((-85.38346 32.64838, -85.38301 ...",0.215017,0.433962,381,0.254593,803,0.171856,0.200498,0.164384,0.077210,0.016032,0.019925,0.054795,0.068259
2,01,081,041700,4,010810417004,AL,Lee County,H1,"MULTIPOLYGON (((-85.37139 32.60139, -85.37138 ...",0.343195,0.225352,1076,0.023234,2745,0.044809,0.331512,0.274681,0.042623,0.041243,0.089253,0.136612,0.102564
3,01,073,011107,4,010730111074,AL,Jefferson County,H1,"MULTIPOLYGON (((-86.64797 33.59205, -86.64771 ...",0.473154,0.064824,1376,0.038517,3377,0.042938,0.350015,0.137104,0.060409,0.005764,0.024578,0.027539,0.022651
4,01,073,011108,4,010730111084,AL,Jefferson County,H1,"MULTIPOLYGON (((-86.65206 33.59869, -86.65204 ...",0.405117,0.151515,543,0.878453,823,0.249089,0.385176,0.069259,0.115431,0.110930,0.000000,0.126367,0.070362
5,01,073,005101,3,010730051013,AL,Jefferson County,H1,"MULTIPOLYGON (((-86.83525 33.49175, -86.83524 ...",0.071823,0.000000,414,0.388889,504,0.099206,0.359127,0.170635,0.075397,0.267647,0.180556,0.180556,0.149171
6,01,073,005101,1,010730051011,AL,Jefferson County,H1,"MULTIPOLYGON (((-86.83960 33.49368, -86.83959 ...",0.041667,0.000000,311,0.929260,576,0.180556,0.046875,0.157986,0.041667,0.000000,0.000000,0.000000,0.000000
7,01,073,005103,1,010730051031,AL,Jefferson County,H1,"MULTIPOLYGON (((-86.83223 33.49198, -86.83099 ...",0.107143,0.000000,572,0.054196,662,0.052870,0.140483,0.377644,0.255287,0.079268,0.039275,0.039275,0.021978
8,01,073,004902,2,010730049022,AL,Jefferson County,H1,"MULTIPOLYGON (((-86.80738 33.49536, -86.80602 ...",0.201232,0.290323,574,0.843206,1074,0.316574,0.266294,0.142458,0.029795,0.041514,0.010242,0.116387,0.059548
9,01,073,011001,4,010730110014,AL,Jefferson County,H1,"MULTIPOLYGON (((-86.56840 33.54323, -86.56821 ...",0.189189,0.061983,412,0.313107,789,0.088720,0.197719,0.326996,0.107731,0.000000,0.000000,0.000000,0.000000


In [ ]:
#output geometry spatially joined with model output
for index, row in FIPS_short.iterrows():
    # print(row['Name'], row['Code'],row['FIPS'])
    ST=FIPS_short.Code[index]
    fps=FIPS_short.FIPS[index]
    print(ST)
    ST_geos=cbg_geos[cbg_geos['State']==ST]
    ST_shp = ST_geos
    print(filename)                   
    print(ST)
    fpth= filename+ "/"+ ST
    ST_shp.to_file(driver = 'ESRI Shapefile', filename= fpth)  